In [1]:
%matplotlib inline


LDA Model
=========

Introduces Gensim's LDA model and demonstrates its use on the NIPS corpus.




In [1]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

The purpose of this tutorial is to demonstrate how to train and tune an LDA model.

In this tutorial we will:

* Load input data.
* Pre-process that data.
* Transform documents into bag-of-words vectors.
* Train an LDA model.

This tutorial will **not**:

* Explain how Latent Dirichlet Allocation works
* Explain how the LDA model performs inference
* Teach you all the parameters and options for Gensim's LDA implementation

If you are not familiar with the LDA model or how to use it in Gensim, I (Olavur Mortensen)
suggest you read up on that before continuing with this tutorial. Basic
understanding of the LDA model should suffice. Examples:

* `Introduction to Latent Dirichlet Allocation <http://blog.echen.me/2011/08/22/introduction-to-latent-dirichlet-allocation>`_
* Gensim tutorial: `sphx_glr_auto_examples_core_run_topics_and_transformations.py`
* Gensim's LDA model API docs: :py:class:`gensim.models.LdaModel`

I would also encourage you to consider each step when applying the model to
your data, instead of just blindly applying my solution. The different steps
will depend on your data and possibly your goal with the model.

Data
----

I have used a corpus of NIPS papers in this tutorial, but if you're following
this tutorial just to learn about LDA I encourage you to consider picking a
corpus on a subject that you are familiar with. Qualitatively evaluating the
output of an LDA model is challenging and can require you to understand the
subject matter of your corpus (depending on your goal with the model).

NIPS (Neural Information Processing Systems) is a machine learning conference
so the subject matter should be well suited for most of the target audience
of this tutorial.  You can download the original data from Sam Roweis'
`website <http://www.cs.nyu.edu/~roweis/data.html>`_.  The code below will
also do that for you.

.. Important::
    The corpus contains 1740 documents, and not particularly long ones.
    So keep in mind that this tutorial is not geared towards efficiency, and be
    careful before applying the code to a large dataset.




In [2]:
import io
import os.path
import re
import tarfile

import smart_open

def extract_documents(url='https://cs.nyu.edu/~roweis/data/nips12raw_str602.tgz'):
    fname = url.split('/')[-1]

    # Download the file to local storage first.
    # We can't read it on the fly because of
    # https://github.com/RaRe-Technologies/smart_open/issues/331
    if not os.path.isfile(fname):
        with smart_open.open(url, "rb") as fin:
            with smart_open.open(fname, 'wb') as fout:
                while True:
                    buf = fin.read(io.DEFAULT_BUFFER_SIZE)
                    if not buf:
                        break
                    fout.write(buf)

    with tarfile.open(fname, mode='r:gz') as tar:
        # Ignore directory entries, as well as files like README, etc.
        files = [
            m for m in tar.getmembers()
            if m.isfile() and re.search(r'nipstxt/nips\d+/\d+\.txt', m.name)
        ]
        for member in sorted(files, key=lambda x: x.name):
            member_bytes = tar.extractfile(member).read()
            yield member_bytes.decode('utf-8', errors='replace')

docs = list(extract_documents())

So we have a list of 1740 documents, where each document is a Unicode string.
If you're thinking about using your own corpus, then you need to make sure
that it's in the same format (list of Unicode strings) before proceeding
with the rest of this tutorial.




In [3]:
print(len(docs))
print(docs[0][:500])

1740
1 
CONNECTIVITY VERSUS ENTROPY 
Yaser S. Abu-Mostafa 
California Institute of Technology 
Pasadena, CA 91125 
ABSTRACT 
How does the connectivity of a neural network (number of synapses per 
neuron) relate to the complexity of the problems it can handle (measured by 
the entropy)? Switching theory would suggest no relation at all, since all Boolean 
functions can be implemented using a circuit with very low connectivity (e.g., 
using two-input NAND gates). However, for a network that learns a pr


Pre-process and vectorize the documents
---------------------------------------

As part of preprocessing, we will:

* Tokenize (split the documents into tokens).
* Lemmatize the tokens.
* Compute bigrams.
* Compute a bag-of-words representation of the data.

First we tokenize the text using a regular expression tokenizer from NLTK. We
remove numeric tokens and tokens that are only a single character, as they
don't tend to be useful, and the dataset contains a lot of them.

.. Important::

   This tutorial uses the nltk library for preprocessing, although you can
   replace it with something else if you want.




In [4]:
# Tokenize the documents.
from nltk.tokenize import RegexpTokenizer

# Split the documents into tokens.
tokenizer = RegexpTokenizer(r'\w+')
for idx in range(len(docs)):
    docs[idx] = docs[idx].lower()  # Convert to lowercase.
    docs[idx] = tokenizer.tokenize(docs[idx])  # Split into words.

# Remove numbers, but not words that contain numbers.
docs = [[token for token in doc if not token.isnumeric()] for doc in docs]

# Remove words that are only one character.
docs = [[token for token in doc if len(token) > 1] for doc in docs]

We use the WordNet lemmatizer from NLTK. A lemmatizer is preferred over a
stemmer in this case because it produces more readable words. Output that is
easy to read is very desirable in topic modelling.




In [5]:
# Lemmatize the documents.
from nltk.stem.wordnet import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()
docs = [[lemmatizer.lemmatize(token) for token in doc] for doc in docs]

LookupError: 
**********************************************************************
  Resource [93mwordnet[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('wordnet')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mcorpora/wordnet[0m

  Searched in:
    - 'C:\\Users\\serge/nltk_data'
    - 'c:\\users\\serge\\appdata\\local\\programs\\python\\python37\\nltk_data'
    - 'c:\\users\\serge\\appdata\\local\\programs\\python\\python37\\share\\nltk_data'
    - 'c:\\users\\serge\\appdata\\local\\programs\\python\\python37\\lib\\nltk_data'
    - 'C:\\Users\\serge\\AppData\\Roaming\\nltk_data'
    - 'C:\\nltk_data'
    - 'D:\\nltk_data'
    - 'E:\\nltk_data'
**********************************************************************


We find bigrams in the documents. Bigrams are sets of two adjacent words.
Using bigrams we can get phrases like "machine_learning" in our output
(spaces are replaced with underscores); without bigrams we would only get
"machine" and "learning".

Note that in the code below, we find bigrams and then add them to the
original data, because we would like to keep the words "machine" and
"learning" as well as the bigram "machine_learning".

.. Important::
    Computing n-grams of large dataset can be very computationally
    and memory intensive.




In [ ]:
# Compute bigrams.
from gensim.models import Phrases

# Add bigrams and trigrams to docs (only ones that appear 20 times or more).
bigram = Phrases(docs, min_count=20)
for idx in range(len(docs)):
    for token in bigram[docs[idx]]:
        if '_' in token:
            # Token is a bigram, add to document.
            docs[idx].append(token)

We remove rare words and common words based on their *document frequency*.
Below we remove words that appear in less than 20 documents or in more than
50% of the documents. Consider trying to remove words only based on their
frequency, or maybe combining that with this approach.




In [6]:
# Remove rare and common tokens.
from gensim.corpora import Dictionary

# Create a dictionary representation of the documents.
dictionary = Dictionary(docs)

# Filter out words that occur less than 20 documents, or more than 50% of the documents.
dictionary.filter_extremes(no_below=20, no_above=0.5)

2022-02-09 03:22:36,699 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2022-02-09 03:22:38,601 : INFO : built Dictionary(79979 unique tokens: ['0a', '2h', '2h2', '2he', '2n']...) from 1740 documents (total 4629808 corpus positions)
2022-02-09 03:22:38,601 : INFO : Dictionary lifecycle event {'msg': "built Dictionary(79979 unique tokens: ['0a', '2h', '2h2', '2he', '2n']...) from 1740 documents (total 4629808 corpus positions)", 'datetime': '2022-02-09T03:22:38.601699', 'gensim': '4.1.2', 'python': '3.7.9 (tags/v3.7.9:13c94747c7, Aug 17 2020, 18:58:18) [MSC v.1900 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.19041-SP0', 'event': 'created'}
2022-02-09 03:22:38,683 : INFO : discarding 72622 tokens: [('0a', 19), ('2h', 16), ('2h2', 1), ('2he', 3), ('__c', 2), ('_k', 6), ('about', 1058), ('abstract', 1740), ('after', 1087), ('alently', 2)]...
2022-02-09 03:22:38,684 : INFO : keeping 7357 tokens which were in no less than 20 and no more than 870 (=50.0%) documents
2022-02-09

Finally, we transform the documents to a vectorized form. We simply compute
the frequency of each word, including the bigrams.




In [7]:
# Bag-of-words representation of the documents.
corpus = [dictionary.doc2bow(doc) for doc in docs]

Let's see how many tokens and documents we have to train on.




In [8]:
print('Number of unique tokens: %d' % len(dictionary))
print('Number of documents: %d' % len(corpus))

Number of unique tokens: 7357
Number of documents: 1740


Training
--------

We are ready to train the LDA model. We will first discuss how to set some of
the training parameters.

First of all, the elephant in the room: how many topics do I need? There is
really no easy answer for this, it will depend on both your data and your
application. I have used 10 topics here because I wanted to have a few topics
that I could interpret and "label", and because that turned out to give me
reasonably good results. You might not need to interpret all your topics, so
you could use a large number of topics, for example 100.

``chunksize`` controls how many documents are processed at a time in the
training algorithm. Increasing chunksize will speed up training, at least as
long as the chunk of documents easily fit into memory. I've set ``chunksize =
2000``, which is more than the amount of documents, so I process all the
data in one go. Chunksize can however influence the quality of the model, as
discussed in Hoffman and co-authors [2], but the difference was not
substantial in this case.

``passes`` controls how often we train the model on the entire corpus.
Another word for passes might be "epochs". ``iterations`` is somewhat
technical, but essentially it controls how often we repeat a particular loop
over each document. It is important to set the number of "passes" and
"iterations" high enough.

I suggest the following way to choose iterations and passes. First, enable
logging (as described in many Gensim tutorials), and set ``eval_every = 1``
in ``LdaModel``. When training the model look for a line in the log that
looks something like this::

   2016-06-21 15:40:06,753 - gensim.models.ldamodel - DEBUG - 68/1566 documents converged within 400 iterations

If you set ``passes = 20`` you will see this line 20 times. Make sure that by
the final passes, most of the documents have converged. So you want to choose
both passes and iterations to be high enough for this to happen.

We set ``alpha = 'auto'`` and ``eta = 'auto'``. Again this is somewhat
technical, but essentially we are automatically learning two parameters in
the model that we usually would have to specify explicitly.




In [9]:
# Train LDA model.
from gensim.models import LdaModel

# Set training parameters.
num_topics = 10
chunksize = 2000
passes = 20
iterations = 400
eval_every = None  # Don't evaluate model perplexity, takes too much time.

# Make a index to word dictionary.
temp = dictionary[0]  # This is only to "load" the dictionary.
id2word = dictionary.id2token

model = LdaModel(
    corpus=corpus,
    id2word=id2word,
    chunksize=chunksize,
    alpha='auto',
    eta='auto',
    iterations=iterations,
    num_topics=num_topics,
    passes=passes,
    eval_every=eval_every
)

2022-02-09 03:22:51,986 : INFO : using autotuned alpha, starting with [0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1]
2022-02-09 03:22:51,987 : INFO : using serial LDA version on this node
2022-02-09 03:22:51,993 : INFO : running online (multi-pass) LDA training, 10 topics, 20 passes over the supplied corpus of 1740 documents, updating model once every 1740 documents, evaluating perplexity every 0 documents, iterating 400x with a convergence threshold of 0.001000
2022-02-09 03:22:51,994 : INFO : PROGRESS: pass 0, at document #1740/1740
2022-02-09 03:22:59,476 : INFO : optimized alpha [0.0618762, 0.05441911, 0.08955546, 0.08055319, 0.083998755, 0.07963415, 0.107414044, 0.06615326, 0.055015083, 0.07736762]
2022-02-09 03:22:59,481 : INFO : topic #1 (0.054): 0.005*"units" + 0.005*"generalization" + 0.004*"hidden" + 0.004*"task" + 0.003*"noise" + 0.003*"layer" + 0.003*"visual" + 0.003*"image" + 0.003*"unit" + 0.002*"approximation"
2022-02-09 03:22:59,482 : INFO : topic #8 (0.055): 0.005*

2022-02-09 03:23:16,031 : INFO : topic #6 (0.054): 0.005*"let" + 0.005*"theorem" + 0.005*"points" + 0.004*"units" + 0.004*"class" + 0.004*"threshold" + 0.004*"matrix" + 0.003*"approximation" + 0.003*"layer" + 0.003*"memory"
2022-02-09 03:23:16,032 : INFO : topic #5 (0.058): 0.011*"recognition" + 0.007*"speech" + 0.006*"image" + 0.005*"classification" + 0.005*"feature" + 0.005*"class" + 0.005*"features" + 0.005*"images" + 0.005*"word" + 0.004*"hidden"
2022-02-09 03:23:16,032 : INFO : topic #3 (0.061): 0.006*"hidden" + 0.006*"gaussian" + 0.004*"units" + 0.004*"variables" + 0.004*"likelihood" + 0.004*"noise" + 0.004*"density" + 0.004*"bayesian" + 0.004*"mixture" + 0.004*"log"
2022-02-09 03:23:16,032 : INFO : topic diff=0.190224, rho=0.377964
2022-02-09 03:23:16,037 : INFO : PROGRESS: pass 6, at document #1740/1740
2022-02-09 03:23:18,536 : INFO : optimized alpha [0.034484427, 0.03724578, 0.041408136, 0.0603381, 0.046886623, 0.058079254, 0.052834433, 0.03558515, 0.04204885, 0.04488613]
202

2022-02-09 03:23:30,376 : INFO : topic #0 (0.031): 0.011*"noise" + 0.009*"generalization" + 0.008*"signal" + 0.006*"examples" + 0.005*"line" + 0.005*"student" + 0.005*"classifier" + 0.005*"teacher" + 0.005*"classifiers" + 0.005*"detection"
2022-02-09 03:23:30,376 : INFO : topic #7 (0.033): 0.010*"control" + 0.007*"image" + 0.007*"analog" + 0.006*"circuit" + 0.004*"chip" + 0.004*"auditory" + 0.004*"sound" + 0.004*"signal" + 0.004*"target" + 0.004*"motor"
2022-02-09 03:23:30,376 : INFO : topic #6 (0.052): 0.007*"let" + 0.006*"theorem" + 0.005*"class" + 0.005*"points" + 0.005*"threshold" + 0.004*"bound" + 0.004*"xi" + 0.004*"approximation" + 0.004*"examples" + 0.004*"dimension"
2022-02-09 03:23:30,377 : INFO : topic #5 (0.059): 0.012*"recognition" + 0.008*"speech" + 0.006*"image" + 0.006*"classification" + 0.006*"feature" + 0.005*"word" + 0.005*"class" + 0.005*"images" + 0.005*"features" + 0.004*"layer"
2022-02-09 03:23:30,378 : INFO : topic #3 (0.060): 0.007*"gaussian" + 0.005*"likelihoo

2022-02-09 03:23:41,035 : INFO : topic #3 (0.063): 0.008*"gaussian" + 0.006*"likelihood" + 0.005*"density" + 0.005*"variables" + 0.005*"mixture" + 0.005*"bayesian" + 0.005*"log" + 0.004*"prior" + 0.004*"estimate" + 0.004*"em"
2022-02-09 03:23:41,035 : INFO : topic diff=0.106410, rho=0.235702
2022-02-09 03:23:41,040 : INFO : PROGRESS: pass 17, at document #1740/1740
2022-02-09 03:23:43,103 : INFO : optimized alpha [0.031473096, 0.04737757, 0.036179803, 0.06353467, 0.04363441, 0.06231046, 0.05439845, 0.033296417, 0.04413693, 0.049396224]
2022-02-09 03:23:43,108 : INFO : topic #0 (0.031): 0.013*"noise" + 0.010*"generalization" + 0.008*"signal" + 0.006*"examples" + 0.005*"line" + 0.005*"student" + 0.005*"teacher" + 0.005*"detection" + 0.005*"classifier" + 0.005*"ensemble"
2022-02-09 03:23:43,108 : INFO : topic #7 (0.033): 0.010*"control" + 0.009*"image" + 0.008*"analog" + 0.006*"circuit" + 0.006*"chip" + 0.005*"signal" + 0.005*"motor" + 0.004*"feedback" + 0.004*"auditory" + 0.004*"sound"
2

We can compute the topic coherence of each topic. Below we display the
average topic coherence and print the topics in order of topic coherence.

Note that we use the "Umass" topic coherence measure here (see
:py:func:`gensim.models.ldamodel.LdaModel.top_topics`), Gensim has recently
obtained an implementation of the "AKSW" topic coherence measure (see
accompanying blog post, http://rare-technologies.com/what-is-topic-coherence/).

If you are familiar with the subject of the articles in this dataset, you can
see that the topics below make a lot of sense. However, they are not without
flaws. We can see that there is substantial overlap between some topics,
others are hard to interpret, and most of them have at least some terms that
seem out of place. If you were able to do better, feel free to share your
methods on the blog at http://rare-technologies.com/lda-training-tips/ !




In [10]:
top_topics = model.top_topics(corpus) #, num_words=20)

# Average topic coherence is the sum of topic coherences of all topics, divided by the number of topics.
avg_topic_coherence = sum([t[1] for t in top_topics]) / num_topics
print('Average topic coherence: %.4f.' % avg_topic_coherence)

from pprint import pprint
pprint(top_topics)

2022-02-09 03:23:47,549 : INFO : CorpusAccumulator accumulated stats from 1000 documents


Average topic coherence: -1.1370.
[([(0.008043829, 'gaussian'),
   (0.005680379, 'likelihood'),
   (0.005267316, 'density'),
   (0.0051350794, 'mixture'),
   (0.005122783, 'variables'),
   (0.005035629, 'bayesian'),
   (0.0047627864, 'log'),
   (0.0045012096, 'prior'),
   (0.0043482003, 'estimate'),
   (0.0042610946, 'em'),
   (0.00416331, 'estimation'),
   (0.004163001, 'variance'),
   (0.0040851105, 'regression'),
   (0.0040287557, 'noise'),
   (0.0039503574, 'approximation'),
   (0.003924611, 'posterior'),
   (0.003731127, 'hidden'),
   (0.0036204944, 'prediction'),
   (0.0033068697, 'matrix'),
   (0.0031702402, 'parameter')],
  -0.8524909398478233),
 ([(0.01399409, 'recognition'),
   (0.008558034, 'speech'),
   (0.006674399, 'feature'),
   (0.0065787453, 'image'),
   (0.0065094875, 'classification'),
   (0.0059827194, 'word'),
   (0.0056412034, 'images'),
   (0.005575261, 'features'),
   (0.0055724564, 'class'),
   (0.003983522, 'distance'),
   (0.003856961, 'layer'),
   (0.0038489

Things to experiment with
-------------------------

* ``no_above`` and ``no_below`` parameters in ``filter_extremes`` method.
* Adding trigrams or even higher order n-grams.
* Consider whether using a hold-out set or cross-validation is the way to go for you.
* Try other datasets.

Where to go from here
---------------------

* Check out a RaRe blog post on the AKSW topic coherence measure (http://rare-technologies.com/what-is-topic-coherence/).
* pyLDAvis (https://pyldavis.readthedocs.io/en/latest/index.html).
* Read some more Gensim tutorials (https://github.com/RaRe-Technologies/gensim/blob/develop/tutorials.md#tutorials).
* If you haven't already, read [1] and [2] (see references).

References
----------

1. "Latent Dirichlet Allocation", Blei et al. 2003.
2. "Online Learning for Latent Dirichlet Allocation", Hoffman et al. 2010.


